In [2]:
%matplotlib agg
from river.datasets import synth
from clustream import CluStream
from river import drift
import math
import numpy as np
import matplotlib.pyplot as plt
from river.utils import dict2numpy
from scipy.spatial import distance_matrix
from time import time
from river import stream
import pandas as pd


In [7]:
#dataset = synth.RandomRBF(seed_model=2,n_classes=2,n_features=2,n_centroids=50)
ds = pd.read_csv('random.csv')
ds.dropna(axis=0,inplace=True)
y = ds.pop('.class')

In [4]:
fig, ax = plt.subplots(nrows=2,ncols=2,figsize = (10,10))
lst1 = []
lst2 = []
lst3 = []
i = 1
k,m = 0,0
#tup = [(0,0),(0,1),(0,2),(1,0),(1,1),(1,2),(2,0),(2,1),(2,2)]
tup = [(0,0),(0,1),(1,0),(1,1)]
l = 0
for x,y in stream.iter_pandas(ds,y):
    
    if y == 1.0 :
        lst1.append([x['X1'],x['X2']])
    elif y == 2.0 :
        lst2.append([x['X1'],x['X2']])
    else :
        lst3.append([x['X1'],x['X2']])

    if i % 600 == 0 and i <= 2400: 
        k,m = tup[l]
        
        lst1 = np.array(lst1)
        lst2 = np.array(lst2) 
        lst3 = np.array(lst3) 
        ax[k,m].scatter(lst1[:,0],lst1[:,1])
        ax[k,m].scatter(lst2[:,0],lst2[:,1])
        ax[k,m].scatter(lst3[:,0],lst3[:,1])
        ax[k,m].set_title(f'T = {i}')
        l += 1
        lst1 = lst1.tolist()
        lst2 = lst2.tolist()
        lst3 = lst3.tolist()

  
    


    

    i += 1 

In [5]:
learner = CluStream(n_macro_clusters=3,max_micro_clusters=20,time_window=500)
test = drift.PageHinkley(30,delta=0.01,threshold=0.05,alpha= 0.9,mode='both')



In [6]:
def get_mean_dist(learner):
    ## Calcule la moyenne des max distances entres les centres des micro clusters
    mc_centers = [dict2numpy(x) for x in dict2numpy(learner.get_micro_clusters())]
    max_dist = np.max(distance_matrix(mc_centers,mc_centers,2),axis=1)
    return np.mean(max_dist)

In [8]:
loop = 0
last_dist = 0
dist = 0
for x,y in stream.iter_pandas(ds,y):
    if loop < 100 :
        learner = learner.learn_one(x)
        
    else :
        if loop ==  2401 :
            break
        #if loop == 500 :
            #print("Done training")
            #learner.offline_cluster()


        learner = learner.learn_one(x)
        dist = get_mean_dist(learner)
        print(dist)
        test.update(dist )
        
        if test.drift_detected : 
            print(f"Drift Detected at instance: {loop}")
        
            
        
    loop += 1
    last_dist = dist



0.3137780016441673
0.31389906706312753
0.31384334069961056
0.3136351829620296
0.313672881038619
0.31360662882608914
0.3136024265838
0.3131755191402866
0.31294394710538787
0.3129719490398536
0.31308207865765403
0.31300729398335053
0.3130224068506629
0.31298335748006034
0.31292640774506447
0.3129257945406787
0.3118365608227009
0.31191300523705723
0.31111090378157236
0.31119159703782207
0.3117959335474214
0.3117994537488096
0.31183694868803224
0.31177135258017413
0.31157999618059534
0.31143091095777453
0.31138008060697425
0.3113055479519145
0.3113709150086859
0.3113975859890706
0.3113997886675638
0.3114016550030739
0.311276032545089
0.311273526591681
0.31116941978305396
0.3110390425670395
0.31097678626982
0.3110782125852062
0.3110585927037727
0.31141327305580757
0.3116372264452289
0.3117097036011252
0.3115557027746019
0.3115507484746114
0.3115317475081957
0.311504053144574
0.3114563613502212
0.31138253453991604
0.31148152472216495
0.31155105361477825
0.3115574025017024
0.31155078656023794

In [9]:
learner.offline_cluster()

CluStream (
  n_macro_clusters=3
  max_micro_clusters=20
  micro_cluster_r_factor=2
  time_window=500
  time_gap=100
  seed=None
)

In [10]:
centers = learner.get_micro_clusters()
centers = dict2numpy(centers)

In [11]:
centers

array([{'X1': 0.24902131179866682, 'X2': 0.6390577354686664},
       {'X1': 0.234129336918095, 'X2': 0.594521428409383},
       {'X1': 0.46455093247193496, 'X2': 0.6462881660015328},
       {'X1': 0.38004468095675686, 'X2': 0.6357500657803292},
       {'X1': 0.247021440623817, 'X2': 0.5770103389638388},
       {'X1': 0.4641934824431406, 'X2': 0.7106024914815724},
       {'X1': 0.5165265987440507, 'X2': 0.665838924082823},
       {'X1': 0.28611606725481725, 'X2': 0.5836003567059963},
       {'X1': 0.259403692923735, 'X2': 0.6136538959207014},
       {'X1': 0.4664518007775516, 'X2': 0.5251834884377311},
       {'X1': 0.2825806441257402, 'X2': 0.6564566251199649},
       {'X1': 0.5163810335125524, 'X2': 0.7383518344219648},
       {'X1': 0.5534907713635111, 'X2': 0.6784010516255818},
       {'X1': 0.3480486863143762, 'X2': 0.6634038134920961},
       {'X1': 0.23379136555033667, 'X2': 0.6165522549661973},
       {'X1': 0.5432993072469656, 'X2': 0.453423056288622},
       {'X1': 0.323062097

In [12]:
centers = [dict2numpy(x) for x in centers]
centers

[array([0.24902131, 0.63905774]),
 array([0.23412934, 0.59452143]),
 array([0.46455093, 0.64628817]),
 array([0.38004468, 0.63575007]),
 array([0.24702144, 0.57701034]),
 array([0.46419348, 0.71060249]),
 array([0.5165266 , 0.66583892]),
 array([0.28611607, 0.58360036]),
 array([0.25940369, 0.6136539 ]),
 array([0.4664518 , 0.52518349]),
 array([0.28258064, 0.65645663]),
 array([0.51638103, 0.73835183]),
 array([0.55349077, 0.67840105]),
 array([0.34804869, 0.66340381]),
 array([0.23379137, 0.61655225]),
 array([0.54329931, 0.45342306]),
 array([0.3230621 , 0.57822729]),
 array([0.28814979, 0.61408204]),
 array([0.54096806, 0.36332334]),
 array([0.31024148, 0.61912903])]

In [ ]:
centers = np.array(centers)

In [13]:
centers = np.array([[0.55930317, 0.32594368],
       [0.46940108, 0.695017  ],
       [0.46644315, 0.52122201]])

In [14]:
ax[1,1].scatter(centers[:,0],centers[:,1])

In [15]:
fig.savefig('concept_drift.jpg')